### For creating points from Latitude and Longitude

In [ ]:
locations = df[['lat', 'lon']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]
# Output: [35.69798843112353, 139.6550454876133]

### For Creating Trajectories with Folium

In [ ]:
import folium

m = folium.Map(location=[34.95, 137.16],
              zoom_start=15)

loc = locationlist

folium.PolyLine(loc,
                color='red',
                weight=1,
                opacity=0.8).add_to(m)
m

### Plot Trajectories over time

Steps:
1. Create Lines for each recurring connections
2. Create Features for each line
3. Plot the features using Folium

#### 1. Create Lines List

In [ ]:
# To append all lines
line_list = []
# Get the starting timestamp
prev_con = max_ssid_df.loc[0, 'connecting_at_jst']
# Start from the second connection and loop until the last connection
for current_con in max_ssid_df[1:].connecting_at_jst:
    line_dict = {
        # Get list of all coordinates
        "coordinates": [
            [max_ssid_df[max_ssid_df.connecting_at_jst==prev_con].reset_index().at[0, 'lon'], max_ssid_df[max_ssid_df.connecting_at_jst==prev_con].reset_index().at[0, 'lat']],
            [max_ssid_df[max_ssid_df.connecting_at_jst==current_con].reset_index().at[0, 'lon'], max_ssid_df[max_ssid_df.connecting_at_jst==current_con].reset_index().at[0, 'lat']],
        ],
        "dates": [prev_con.strftime('%Y-%m-%d %H:%M:%S'), current_con.strftime('%Y-%m-%d %H:%M:%S')],
        "color": date_color_map[max_ssid_df[max_ssid_df.connecting_at_jst==prev_con].reset_index().at[0, 'date']],
    }
    line_list.append(line_dict)
    # Set previous time as current time for next iteration
    prev_con=current_con
print(line_list)

#### 2. Initialize Features

In [ ]:
# Create features
features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "LineString",
            "coordinates": line["coordinates"],
        },
        "properties": {  
            "times": line["dates"],
            "style": {
                "color": line["color"],
                "weight": line["weight"] if "weight" in line else 5,
            },
        },
    }
    for line in line_list
]

#### 3. Plot Trajectories using Folium

In [ ]:
import folium
from branca.element import Figure
from folium.plugins import TimestampedGeoJson
fig=Figure(width=850,height=550)
m=folium.Map(location=[34.95, 137.16],zoom_start=13)


TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": features,
    },
    period="PT1M",
    add_last_point=False,
).add_to(m)
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Water Color').add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
folium.LayerControl().add_to(m)
m.save(f"{OUT_PATH}eda/movement/ssid/user_id_{''.join(max_user_id.split('/'))}_nationwide_movement_trajectory_every_second.html")
#mdate

### Plot Trajectories By Date

In [ ]:
import folium
from branca.element import Figure
from folium.plugins import TimestampedGeoJson

def create_reccurring_connections(temp_df):
    # To append all lines
    line_list = []
    # Get the starting timestamp
    prev_con = temp_df.loc[0, 'connecting_at_jst']
    # Start from the second connection and loop until the last connection
    for current_con in temp_df[1:].connecting_at_jst:
        line_dict = {
            # Get list of all coordinates
            "coordinates": [
                [temp_df[temp_df.connecting_at_jst==prev_con].reset_index().at[0, 'lon'], temp_df[temp_df.connecting_at_jst==prev_con].reset_index().at[0, 'lat']],
                [temp_df[temp_df.connecting_at_jst==current_con].reset_index().at[0, 'lon'], temp_df[temp_df.connecting_at_jst==current_con].reset_index().at[0, 'lat']],
            ],
            "dates": [prev_con.strftime('%Y-%m-%d %H:%M:%S'), current_con.strftime('%Y-%m-%d %H:%M:%S')],
            "color": date_color_map[temp_df[temp_df.connecting_at_jst==prev_con].reset_index().at[0, 'date']],
        }
        line_list.append(line_dict)
        # Set previous time as current time for next iteration
        prev_con=current_con
        
    return line_list

def create_features_from_line_list(line_list):
    features = [
        {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": line["coordinates"],
            },
            "properties": {
                "times": line["dates"],
                "style": {
                    "color": line["color"],
                    "weight": line["weight"] if "weight" in line else 5,
                },
            },
        }
        for line in line_list
    ]
    
    return features

def create_map(features):
    
    fig=Figure(width=850,height=550)
    # Initialize the Map
    m=folium.Map(location=[34.95, 137.16],zoom_start=13)
    # Create subgroup
    #g1 = folium.plugins.FeatureGroupSubGroup(m, str(date))
    TimestampedGeoJson(
        {
            "type": "FeatureCollection",
            "features": features,
        },
        period="PT1M",
        add_last_point=False,
    ).add_to(m)
    folium.TileLayer('Stamen Terrain').add_to(m)
    folium.TileLayer('Stamen Toner').add_to(m)
    folium.TileLayer('Stamen Water Color').add_to(m)
    folium.TileLayer('cartodbpositron').add_to(m)
    folium.TileLayer('cartodbdark_matter').add_to(m)
    folium.LayerControl().add_to(m)
    
    return m


# Add starting date
start_date=max_ssid_df.date.sort_values().unique()[0]
# Iterate over all dates
for date in max_ssid_df.date.sort_values().unique():
    
    temp_df = max_ssid_df[max_ssid_df.date==date].copy()
    
    # If it's not first date then reset the index
    if date != start_date:
        temp_df = temp_df.reset_index(drop=True)
    # Sort dataframe by Connection time
    temp_df = temp_df.sort_values(by='connecting_at_jst', ascending=True)
    print(temp_df.shape)
    
    temp_df.to_csv(f"{OUT_PATH}eda/movement/ssid/user_id_{max_user_id}_all_{date}_nationwide_movement_df.csv", index=False)
    
    
    line_list = create_reccurring_connections(temp_df)
    print(len(line_list))
    
    features = create_features_from_line_list(line_list)
    print(len(features))
    
    map_obj = create_map(features)
    #break
    map_obj.save(f"{OUT_PATH}eda/movement/ssid/user_id_{max_user_id}_all_{date}_nationwide_movement_trajectory_every_second.html")
    #m